## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-25-23-01-51 +0000,nyt,Live Updates: Cuban Government Says Troops Kil...,https://www.nytimes.com/live/2026/02/25/world/...
1,2026-02-25-23-01-50 +0000,nypost,"Vermont elementary school principal, 41, on pa...",https://nypost.com/2026/02/25/us-news/vermont-...
2,2026-02-25-22-58-48 +0000,cbc,Harvey Weinstein hires Luigi Mangione's and Se...,https://www.cbc.ca/news/entertainment/weinstei...
3,2026-02-25-22-57-54 +0000,nyt,"TSA PreCheck Is Back, but Global Entry Is Paus...",https://www.nytimes.com/2026/02/25/travel/tsa-...
4,2026-02-25-22-57-00 +0000,wsj,Former Star Supreme Court Lawyer Convicted of ...,https://www.wsj.com/us-news/law/former-star-su...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2555/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
66,trump,84
201,union,33
356,new,16
175,iran,15
718,will,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
322,2026-02-25-05-07-00 +0000,wsj,President Trump told a national audience in hi...,https://www.wsj.com/politics/policy/trump-hail...,206
327,2026-02-25-04-57-00 +0000,wsj,President Trump delivered a State of the Union...,https://www.wsj.com/politics/policy/five-takea...,161
205,2026-02-25-15-13-00 +0000,wsj,Didn’t catch all of President Trump’s State of...,https://www.wsj.com/politics/policy/trump-stat...,161
329,2026-02-25-04-39-22 +0000,cbc,Trump honours Winnipeg Jets goalie with Presid...,https://www.cbc.ca/news/canada/manitoba/helleb...,153
108,2026-02-25-20-13-37 +0000,nypost,Rivals roast ‘tired’ Rep. Tom Suozzi for dozin...,https://nypost.com/2026/02/25/us-news/rivals-r...,149


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
322,206,2026-02-25-05-07-00 +0000,wsj,President Trump told a national audience in hi...,https://www.wsj.com/politics/policy/trump-hail...
349,103,2026-02-25-03-24-12 +0000,nyt,“Under my just-enacted most favored nation agr...,https://www.nytimes.com/live/2026/02/25/us/fac...
331,94,2026-02-25-04-06-50 +0000,nyt,“They’ve killed and maimed thousands of Americ...,https://www.nytimes.com/live/2026/02/25/us/fac...
344,78,2026-02-25-03-35-45 +0000,nyt,"“With the great Big Beautiful Bill, we gave yo...",https://www.nytimes.com/live/2026/02/25/us/fac...
34,70,2026-02-25-22-28-00 +0000,wsj,Spy agency blocks Congress from seeing Gabbard...,https://www.wsj.com/politics/policy/spy-agency...
336,54,2026-02-25-03-56-16 +0000,nyt,"“In a breakthrough operation last June, the Un...",https://www.nytimes.com/live/2026/02/25/us/fac...
144,53,2026-02-25-18-29-00 +0000,wsj,"Casey Means, President Trump’s surgeon-general...",https://www.wsj.com/politics/policy/surgeon-ge...
380,50,2026-02-24-23-54-43 +0000,nypost,Warner Bros. Discovery may upend Netflix deal ...,https://nypost.com/2026/02/24/media/warner-bro...
343,47,2026-02-25-03-35-53 +0000,nyt,“I want to stop all payments to big insurance ...,https://www.nytimes.com/live/2026/02/24/us/tru...
157,46,2026-02-25-17-17-41 +0000,latimes,FBI raids of LAUSD Supt. Alberto Carvalho's ho...,https://www.latimes.com/california/story/2026-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
